<h3> Reto: Crear la Clase ConBacktester desde Cero</h3>
Pasos:  
1. Inicializar las propiedades symbol, start, end, tc y results.  
2. Agregar el método get_data() para recuperar y preparar los datos crudos del archivo "intraday_pairs.csv". Llamar a get_data() dentro del método "dunder init"
3. Agregar el método test_strategy() con el parámetro window
4. Agregar el método plot_results().
5. Agregar una representación adecuada y Docstrings.

## Instanciación

In [1]:
class ConBacktester():
    def __init__(self, symbol, start, end, tc):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.tc = tc
        self.results = None


In [2]:
tester = ConBacktester("EURUSD", "2018-01-01", "2019-12-31", 0)

In [3]:
tester.end

'2019-12-31'

In [4]:
tester.tc

0

## Método get_data()